In [1]:
import tensorflow.compat.v1 as tf 
import tensorflow.keras as keras
import pandas as pd
import numpy as np
tf.compat.v1.disable_eager_execution()

In [9]:
def normalize(dataset):
    dataset = dataset.apply(lambda x: (x - x.min(axis = 0)) / (x.max(axis = 0) - x.min(axis = 0)))
    dataset = dataset.loc[:,dataset.any()]
    return dataset

In [10]:
dataset = pd.read_csv("2018.csv" , index_col = 0)
# dataset = dataset[['DAY', 'MONTH' ,'DEP_TIME','ARR_TIME','DISTANCE','ARR_DELAY']]
dataset = normalize(dataset.iloc[:,:])
# Splitting the dataset into train 70% and test 30%
features = (dataset.shape[1] - 1)
examples = dataset.shape[0]
train_size = int (0.7 * len(dataset))
(x_train , y_train) = (np.array(dataset.iloc[:train_size,:features]), np.array(dataset.iloc[:train_size,features:]))
(x_test , y_test) = (np.array(dataset.iloc[train_size:,:features]), np.array(dataset.iloc[train_size:,features:]))
dataset.head(10)

,DAY,MONTH,DISTANCE,DEP_TIME,ARR_TIME,ORIGIN_ABE,ORIGIN_ABI,ORIGIN_ABQ,ORIGIN_ABR,ORIGIN_ABY,...,DEST_USA,DEST_VEL,DEST_VLD,DEST_VPS,DEST_WRG,DEST_WYS,DEST_XNA,DEST_YAK,DEST_YUM,ARR_DELAY
0,0.066667,0.272727,0.166195,0.709462,0.760734,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.266667,1.000000,0.131058,0.435181,0.474364,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.133333,0.181818,0.041397,0.222176,0.263026,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.666667,0.363636,0.059370,0.507295,0.549812,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.166667,0.272727,0.021002,0.550646,0.585244,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,0.333333,0.545455,0.068457,0.228012,0.300542,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
6,0.000000,0.272727,0.265347,0.762401,0.837849,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
7,0.733333,0.090909,0.080775,0.726553,0.807003,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
8,0.100000,0.363636,0.251414,0.472697,0.683201,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
9,0.533333,0.818182,0.061793,0.273864,0.337641,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [11]:
# hyper parameters 
learning_rate = 0.01
epochs = 50
batch_size = 100
batches = int (train_size / batch_size)

In [ ]:
(hidden1_size, hidden2_size) = (100, 50)
x = tf.placeholder(tf.float32 , [None , features])
y = tf.placeholder(tf.float32 , [None , 1])

W = tf.Variable(tf.zeros([features, 1]))
b = tf.Variable(tf.zeros([1]))


W1 = tf.Variable(tf.truncated_normal([features, hidden1_size], stddev=0.1))
b1 = tf.Variable(tf.constant(0.1, shape=[hidden1_size]))
z1 = tf.nn.relu(tf.matmul(x,W1)+b1)
W2 = tf.Variable(tf.truncated_normal([hidden1_size, hidden2_size], stddev=0.1))
b2 = tf.Variable(tf.constant(0.1, shape=[hidden2_size]))
z2 = tf.nn.relu(tf.matmul(z1,W2)+b2)
W3 = tf.Variable(tf.truncated_normal([hidden2_size, 1], stddev=0.1))
b3 = tf.Variable(tf.constant(0.1, shape=[1]))


pred = 1 / (1.0 + tf.exp(-z2))
loss = tf.reduce_mean(-(y * tf.log(pred) + (1 - y) * tf.log( 1 - pred)))
update = tf.train.GradientDescentOptimizer(0.001).minimize(loss)





# pred = tf.nn.sigmoid(tf.matmul(z2,W3) + b3)  
# loss = tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(labels = y, logits = pred))
# update = tf.train.GradientDescentOptimizer(learning_rate).minimize(loss)
# Define accuracy - Percentage of predictions did we get right
predicted_class = tf.greater(pred,0.5)
correct = tf.equal(predicted_class, tf.equal(y,1.0))
accuracy = tf.reduce_mean(tf.cast(correct,tf.float32))
# correct_prediction = tf.equal(tf.argmax(pred, 1),tf.argmax(y, 1)) 
# accuracy = tf.reduce_mean(tf.cast(correct_prediction,tf.float32))

session = tf.Session()
session.run(tf.global_variables_initializer())

for epoch in range(10000):     
    for row in range(batches):        
        batch_x = x_train [row * batch_size : (row + 1) * batch_size]
        batch_y = y_train [row * batch_size : (row + 1) * batch_size]
        session.run(update ,feed_dict = {x : batch_x , y : batch_y})
        _, loss_val = session.run([update, loss],feed_dict={x: batch_x, y: batch_y})
        
    if(epoch%100 == 0):    
        print("Iteration: ",epoch, " Accuracy:", session.run(accuracy, feed_dict={x: x_test, y: y_test}),"loss:",loss_val)

Iteration:  0  Accuracy: 0.45381254 loss: 0.70785177
Iteration:  100  Accuracy: 0.6169355 loss: 0.6933933
Iteration:  200  Accuracy: 0.62429863 loss: 0.6932423
Iteration:  300  Accuracy: 0.62616014 loss: 0.69320935
Iteration:  400  Accuracy: 0.6269072 loss: 0.69319385
Iteration:  500  Accuracy: 0.6273 loss: 0.69318473
Iteration:  600  Accuracy: 0.6276552 loss: 0.6931787
Iteration:  700  Accuracy: 0.62783986 loss: 0.6931729
Iteration:  800  Accuracy: 0.6279708 loss: 0.69316876
Iteration:  900  Accuracy: 0.628049 loss: 0.69316554
Iteration:  1000  Accuracy: 0.628146 loss: 0.6931627
Iteration:  1100  Accuracy: 0.6282393 loss: 0.69316006
Iteration:  1200  Accuracy: 0.628293 loss: 0.69315773
Iteration:  1300  Accuracy: 0.6283457 loss: 0.6931557
Iteration:  1400  Accuracy: 0.628374 loss: 0.69315416
Iteration:  1500  Accuracy: 0.62840223 loss: 0.69315267
Iteration:  1600  Accuracy: 0.6284173 loss: 0.69315153
Iteration:  1700  Accuracy: 0.6284494 loss: 0.69315046
Iteration:  1800  Accuracy: 0.